In [165]:
import pandas as pd
import random
import numpy as np
import glob
from collections import Counter
from collections import defaultdict

# This document generates sample sets to use as testable iterations for imputation accuracy

> we start with 75 animals availabe on all 3 chips

> next we get a subset of all angus animals  that doesn't overlap with this test set for each chip so we're not working with the full set every time (might change)

> then we'll make a list of each of these full angus sets) that fits the plink standard

> then we'll make a couple of random subsets of the 75 testers. 

In [3]:
! ls ../../GEN_DUMP/

227234.161117.12083.100_B.ID   58336.161117.7744.100_A.ID
227234.161117.12083.100_B.ped  58336.161117.7744.100_A.ped
227234.161117.1994.200_A.ID    777962.161117.1681.100_A.ID
227234.161117.1994.200_A.ped   777962.161117.1681.100_A.ped
58336.161117.1062.100_C.ID     777962.161117.219.200_B.ID
58336.161117.1062.100_C.ped    777962.161117.219.200_B.ped
58336.161117.127.100_B.ID      777962.161117.2779.200_A.ID
58336.161117.127.100_B.ped     777962.161117.2779.200_A.ped
58336.161117.1339.200_A.ID     777962.161117.417.100_B.ID
58336.161117.1339.200_A.ped    777962.161117.417.100_B.ped
58336.161117.34296.200_B.ID    DUMP_GEN_for_imputation.sql
58336.161117.34296.200_B.ped   process_gen_dump.txt


In [7]:
Angus = glob.glob('../../GEN_DUMP/*.100*.ID')
Angus

['../../GEN_DUMP/227234.161117.12083.100_B.ID',
 '../../GEN_DUMP/58336.161117.1062.100_C.ID',
 '../../GEN_DUMP/58336.161117.127.100_B.ID',
 '../../GEN_DUMP/58336.161117.7744.100_A.ID',
 '../../GEN_DUMP/777962.161117.1681.100_A.ID',
 '../../GEN_DUMP/777962.161117.417.100_B.ID']

In [88]:
def Breedmaker(blist):
    data = []
    for i in blist:
        array,date,count,brV,ftype = i.split('/')[-1].split('.')
        breed,version = brV.split('_')
        data.append([i,array,version,int(count),breed,date])
    return pd.DataFrame(data,columns=['fname','array','version','total','breed','date'])

In [118]:
angframe = Breedmaker(Angus)
perChip=angframe.groupby('array').sum()
print(perChip)

        total
array        
227234  12083
58336    8933
777962   2098


In [111]:
usenow = pd.Series({'227234':2000,'58336':2000,'777962':1000},name='usenow')

In [119]:
perChip = perChip.join(usenow)

In [120]:
angframe = angframe.join(perChip,rsuffix='sum',on='array')

In [121]:
angframe

,fname,array,version,total,breed,date,totalsum,usenow
0,../../GEN_DUMP/227234.161117.12083.100_B.ID,227234,B,12083,100,161117,12083,2000
1,../../GEN_DUMP/58336.161117.1062.100_C.ID,58336,C,1062,100,161117,8933,2000
2,../../GEN_DUMP/58336.161117.127.100_B.ID,58336,B,127,100,161117,8933,2000
3,../../GEN_DUMP/58336.161117.7744.100_A.ID,58336,A,7744,100,161117,8933,2000
4,../../GEN_DUMP/777962.161117.1681.100_A.ID,777962,A,1681,100,161117,2098,1000
5,../../GEN_DUMP/777962.161117.417.100_B.ID,777962,B,417,100,161117,2098,1000


In [123]:
angframe['percentofchip'] = angframe['total'] / angframe.totalsum
angframe['animal_use'] = angframe.percentofchip * angframe.usenow
angframe

,fname,array,version,total,breed,date,totalsum,usenow,percentofchip,animal_use
0,../../GEN_DUMP/227234.161117.12083.100_B.ID,227234,B,12083,100,161117,12083,2000,1.000000,2000.000000
1,../../GEN_DUMP/58336.161117.1062.100_C.ID,58336,C,1062,100,161117,8933,2000,0.118885,237.770066
2,../../GEN_DUMP/58336.161117.127.100_B.ID,58336,B,127,100,161117,8933,2000,0.014217,28.433897
3,../../GEN_DUMP/58336.161117.7744.100_A.ID,58336,A,7744,100,161117,8933,2000,0.866898,1733.796037
4,../../GEN_DUMP/777962.161117.1681.100_A.ID,777962,A,1681,100,161117,2098,1000,0.801239,801.239276
5,../../GEN_DUMP/777962.161117.417.100_B.ID,777962,B,417,100,161117,2098,1000,0.198761,198.760724


In [255]:
angframe

,fname,array,version,total,breed,date,totalsum,usenow,percentofchip,animal_use
0,../../GEN_DUMP/227234.161117.12083.100_B.ID,227234,B,12083,100,161117,12083,2000,1.000000,2000.000000
1,../../GEN_DUMP/58336.161117.1062.100_C.ID,58336,C,1062,100,161117,8933,2000,0.118885,237.770066
2,../../GEN_DUMP/58336.161117.127.100_B.ID,58336,B,127,100,161117,8933,2000,0.014217,28.433897
3,../../GEN_DUMP/58336.161117.7744.100_A.ID,58336,A,7744,100,161117,8933,2000,0.866898,1733.796037
4,../../GEN_DUMP/777962.161117.1681.100_A.ID,777962,A,1681,100,161117,2098,1000,0.801239,801.239276
5,../../GEN_DUMP/777962.161117.417.100_B.ID,777962,B,417,100,161117,2098,1000,0.198761,198.760724


In [138]:
random.sample(range(10),5)

[2, 8, 5, 3, 6]

In [184]:
allanimals = []
perarray = defaultdict(list)
for f in angframe.itertuples():
    print(f[1],f[-1],f[4])
    fname,touse,total,array = f[1],f[-1],f[4],f[2]
    fix = random.sample(range(total),int(touse))
    bix = idgrab(fname)
    perarray[array].append(bix)
    allanimals =allanimals + bix
    
fname,total,touse

../../GEN_DUMP/227234.161117.12083.100_B.ID 2000.0 12083
../../GEN_DUMP/58336.161117.1062.100_C.ID 237.770066047 1062
../../GEN_DUMP/58336.161117.127.100_B.ID 28.4338967872 127
../../GEN_DUMP/58336.161117.7744.100_A.ID 1733.79603717 7744
../../GEN_DUMP/777962.161117.1681.100_A.ID 801.2392755 1681
../../GEN_DUMP/777962.161117.417.100_B.ID 198.7607245 417


('../../GEN_DUMP/777962.161117.417.100_B.ID', 417, 198.76072449952338)

In [193]:
arraynames = {}
for i in perarray:
    arraynames[i] = [val for sublist in perarray[i] for val in sublist]
[len(i) for i in arraynames.values()]
#perarray['58336'][1][:20]

[8933, 12083, 2098]

In [197]:
arraynames = {}
for i in perarray:
    arraynames[i] = set([val for sublist in perarray[i] for val in sublist])
[len(i) for i in arraynames.values()]
animalrecords = {}

for i in arraynames:
    animalrecords[i] = defaultdict(int)
    for t in arraynames[i]:
        animalrecords[i][t]= animalrecords[i][t]+1

In [203]:
chipperanimal = pd.DataFrame(animalrecords)
chipperanimal.shape

(21569, 3)

In [205]:
chipperanimal.sum().sum()

23105.0

In [230]:
chipperanimal.fillna(0,inplace=True)
chipperanimal['sum'] = chipperanimal.sum(axis=1)
chipperanimal.head()

,227234,58336,777962,sum
AANUSAF000017102723,0.0,0.0,1.0,1.0
AANUSAF000017102725,0.0,0.0,1.0,1.0
AANUSAF000017102733,0.0,0.0,1.0,1.0
AANUSAF000017102750,0.0,0.0,1.0,1.0
AANUSAF000017102754,0.0,0.0,1.0,1.0


In [275]:
snp50set = chipperanimal[(chipperanimal['227234'] == 0) & (chipperanimal['58336'] == 1) &(chipperanimal['777962'] == 0)]
f250set = chipperanimal[(chipperanimal['227234'] == 1) & (chipperanimal['58336'] == 0) &(chipperanimal['777962'] == 0)]
hdset = chipperanimal[(chipperanimal['227234'] == 0) & (chipperanimal['58336'] == 0) &(chipperanimal['777962'] == 1)]
f250snp50 = chipperanimal[(chipperanimal['227234'] == 1) & (chipperanimal['58336'] == 1) &(chipperanimal['777962'] == 0)]
f250snp50.sample(20).index

Index(['UMCUSAU000000091543', 'UMCUSAU000000091716', 'UMCUSAF000000108463',
       'UMCUSAU000000091450', 'UMCUSAU000000092139', 'UMCUSAU000000091448',
       'UMCUSAU000000092321', 'UMCUSAU000000091636', 'UMCUSAU000000091844',
       'UMCUSAU000000091986', 'UMCUSAF000000108310', 'UMCUSAU000000092132',
       'UMCUSAU000000091942', 'UMCUSAU000000091703', 'UMCUSAU000000091912',
       'UMCUSAM000000073570', 'UMCUSAU000000091615', 'UMCUSAF000000108193',
       'UMCUSAM000000073693', 'UMCUSAU000000092340'],
      dtype='object')

# This will be used for making random samples of animals who are not on multiple chips
# These animals should all be processed and then merged into one file, they can be pulled out of there. 
# duplicate animals can be merged into seperate file, or kept in their original files
# animals that are f250 and chip should perhaps be prioritized, but with only their f250 data, or with their full complement of data, but not for the animals being tested.  


In [304]:
double = f250snp50.sample(600).index.tolist()
double[:10]

['AANUSAM000013567961',
 'AANUSAM000013009185',
 'AANUSAM000013035015',
 'UMCUSAU000000087844',
 'AANUSAM000013311158',
 'AANUSAM000013505534',
 'UMCUSAM000000076491',
 'UMCUSAM000000080846',
 'AANUSAM000013286230',
 'AANUSAM000009255702']

In [311]:
funcset = f250set.sample(400).index.tolist()

In [314]:
fiftyset = snp50set.sample(1000).index.tolist()
hdout = hdset.sample(1000).index.tolist()

# merge all assays togehter first

In [271]:
chipperanimal[(chipperanimal['sum'] > 1) &(chipperanimal['777962'] == 1) ].describe()

,227234,58336,777962,sum
count,240.000000,240.000000,240.0,240.000000
mean,0.700000,0.587500,1.0,2.287500
std,0.459215,0.493313,0.0,0.453543
min,0.000000,0.000000,1.0,2.000000
25%,0.000000,0.000000,1.0,2.000000
50%,1.000000,1.000000,1.0,2.000000
75%,1.000000,1.000000,1.0,3.000000
max,1.000000,1.000000,1.0,3.000000


In [251]:
threechips = chipperanimal[(chipperanimal['227234'] == 1) & (chipperanimal['58336'] == 1) &(chipperanimal['777962'] == 1)]
threechips.head()
threechips.shape

(69, 4)

In [164]:
print(len(allanimals))
len(set(allanimals))
Counter(allanimals).most_common(10)

23114


[('AANUSAM000013531361', 3),
 ('AANUSAM000012227124', 3),
 ('AANUSAM000014257843', 3),
 ('AANUSAM000014000000', 3),
 ('AANUSAM000010776479', 3),
 ('AANUSAM000008781383', 3),
 ('AANUSAM000011216428', 3),
 ('AANUSAM000008477778', 3),
 ('AANUSAM000015150605', 3),
 ('AANUSAM000013627964', 3)]

In [153]:
def idgrab(idfile):
    temp = []
    with open(idfile) as fp:
        for line in fp:
            name = line.split()[1]
            temp.append(name)
    return temp
print(name)

UMCUSAF000000109651


In [273]:
len(idgrab(fname,set(fix)))

TypeError: idgrab() takes 1 positional argument but 2 were given

In [223]:
bobtest = idgrab('../raw_genotypes/777962.160906.75.imp_test.ID')

['AANUSAM000011393414',
 'AANUSAM000012886399',
 'AANUSAM000012588547',
 'AANUSAM000012363922',
 'AANUSAM000014000000',
 'AANUSAM000011462197',
 'AANUSAM000012502030',
 'AANUSAM000012240991',
 'AANUSAM000013300751',
 'AANUSAM000011994601']

In [245]:
bobanimal = chipperanimal.loc[bobtest]
bobanimal.sort_values(by='sum').head(6)

,227234,58336,777962,sum
AANUSAM000009911822,1.0,1.0,0.0,2.0
AANUSAM000012729973,0.0,1.0,1.0,2.0
AANUSAM000005221298,0.0,1.0,1.0,2.0
AANUSAM000011060761,1.0,1.0,0.0,2.0
AANUSAM000013085417,1.0,1.0,0.0,2.0
AANUSAM000015134182,1.0,1.0,0.0,2.0


In [253]:
! grep 'AANUSAM000005221298' ../../GEN_DUMP/*.100*.ID


../../GEN_DUMP/58336.161117.7744.100_A.ID: 1230 AANUSAM000005221298
../../GEN_DUMP/777962.161117.1681.100_A.ID: 80 AANUSAM000005221298


In [254]:
! grep 'AANUSAM000005221298' ../raw_genotypes/*.ID

../raw_genotypes/227234.160906.75.imp_test.ID: 62 AANUSAM000005221298
../raw_genotypes/58336.160906.75.imp_test.ID: 46 AANUSAM000005221298
../raw_genotypes/777962.160906.100.test_hd_A.ID: 84 AANUSAM000005221298
../raw_genotypes/777962.160906.75.imp_test.ID: 46 AANUSAM000005221298


In [249]:
for f in angframe.itertuples():

True

In [64]:
perChip.loc['227234']

count    12083
Name: 227234, dtype: int64

In [ ]:
for f in angframe.itertuples():
    print(f[1],f[-1],f[4])
    fname,touse,total,array = f[1],f[-1],f[4],f[2]
    fix = random.sample(range(total),int(touse))
    bix = idgrab(fname)
    perarray[array].append(bix)
    allanimals =allanimals + bix